## sin関数をフィッティングしてみる

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

$sin (\pi x)$にノイズを加えたデータを用意する

In [ ]:
x_data = np.float32(np.random.rand(30)) # Random input
y_data = np.sin(2*np.pi*x_data) + np.random.rand()
plt.scatter(x_data, y_data)

$y=w_1*x+w_2*x^2+w_3*x^3+b$のグラフを作る

グラフ内で変数（パラメータ）を作るにはtf.Variable（）を使う   
さらに，わかりやすいように変数にtf.name_scope()で名前をつける

In [ ]:
dim = 3
x = tf.placeholder(tf.float32, [None], 'x')
y = tf.placeholder(tf.float32, [None], 'y')

with tf.name_scope('b'):
    b = tf.Variable(random.random())
out = b
w_list = []
for i in range(1,dim+1):
    with tf.name_scope('w'+str(i)):
        w = tf.Variable(random.random())
    w_list.append(w)
    out = w*x**i+out

グラフの設計時にループを使っても演算(推論，学習)時には完成したグラフで演算するので遅くならない

損失関数を誤差の２乗   
最適化アルゴリズムを最急降下法

In [ ]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(out - y))
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train = optimizer.minimize(loss)

セッションを作って初期化

In [ ]:
sess = tf.Session()
init= tf.global_variables_initializer()
sess.run(init)

Tensorboard用にトラッキングを追加    
今回は，損失関数の値とグラフを保存する

In [ ]:
with tf.name_scope('summary'):
    merged = tf.summary.scalar('loss', loss)
    writer = tf.summary.FileWriter('logs',  graph=sess.graph)
    i = 0

トラッキングしながら学習

In [ ]:
for step in range(0, 1000):
    i+=1
    _, summary = sess.run([train,merged], feed_dict={x:x_data,y:y_data})
    writer.add_summary(summary, i)
print(sess.run(loss, feed_dict={x:x_data,y:y_data}))
plt.scatter(x_data, sess.run(out, feed_dict={x:x_data,y:y_data}))

# tensorboardで学習の可視化
tensorboardの起動   
`$ tensorboard --logdir='logs'`  

[http://127.0.0.1:6006/](http://127.0.0.1:6006/)からtensorboardにアクセス